In [5]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from itertools import product 
import plotly.graph_objects as go

In [259]:
df = pd.read_csv("AGB_2009-2021.csv",header=[0,1]);

In [261]:
# how to select using multi-index
df.loc[1,("C3","2009")];
df[("C3","2009")];

In [262]:
# how to change columns names
df.rename(columns=
          
          {'Unnamed: 0_level_0':'Temperature',
           'Unnamed: 1_level_0':'Precipitation',
           'Unnamed: 2_level_0':'Plot',
          }, 
          
          level=0,inplace=True )

In [289]:
header = ['Temperature', 'Precipitation', 'Plot', 'Year', 'PFT', 'Biomass']
dfx = pd.DataFrame(columns=header)
for i in range(3,df.shape[1]):
    dff1 = pd.DataFrame()
    dff1["Temperature"]   = df[('Temperature', 'Warmed')]
    dff1["Precipitation"] = df[('Precipitation',  'Water')]
    dff1["Plot"]          = df[('Plot','Plot')]
    dff1["Year"]          = df.columns[i][1]
    dff1['PFT']           = df.columns[i][0]
    dff1['Biomass']       = df[df.columns[i]]
    
    dfx = pd.concat([dfx,dff1],'rows')
    
dfx2 = dfx.groupby(["Temperature","Precipitation","PFT","Year"]).mean()  

dfx3 = dfx2.reset_index()
dfx3["Trmt"] = [''.join(dfx2.index[i][0:2]) for i in range(dfx2.index.shape[0])]
dfx3

In [290]:
dfx.head()

,Temperature,Precipitation,Plot,Year,PFT,Biomass
0,C,C,3,2009,C3,251.76
1,C,C,6,2009,C3,415.71
2,C,C,19,2009,C3,429.18
3,C,C,23,2009,C3,504.84
4,C,D,9,2009,C3,276.84


In [291]:
# calculate mean
# dfx1 = dfx[dfx['Year'] == selected_tem]
    
dfx2 = dfx.groupby(["Temperature","Precipitation","PFT","Year"]).mean()  

dfx3 = dfx2.reset_index()
dfx3["Trmt"] = [''.join(dfx2.index[i][0:2]) for i in range(dfx2.index.shape[0])]
dfx3
# dfx3.drop(dfx3[dfx3.PFT=="Total"].index,inplace=True)

,Temperature,Precipitation,PFT,Year,Biomass,Trmt
0,C,C,C3,2009,400.3725,CC
1,C,C,C3,2010,448.1850,CC
2,C,C,C3,2011,157.3075,CC
3,C,C,C3,2012,294.1075,CC
4,C,C,C3,2013,283.8825,CC
...,...,...,...,...,...,...
193,W,H,Total,2015,438.9850,WH
194,W,H,Total,2016,294.6800,WH
195,W,H,Total,2017,208.8500,WH
196,W,H,Total,2020,202.4200,WH


In [3]:
# !!!!!!! dashboard starts here !!!!!!!!!
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from itertools import product 
import plotly.graph_objects as go



In [7]:

avocado = pd.read_csv('avocado-updated-2020.csv')

app = JupyterDash(__name__)
geo_dropdown = dcc.Dropdown(options=avocado['geography'].unique(),
                            value='New York')

app.layout = html.Div(children=[
    html.H1(children='Avocado Prices Dashboard'),
    geo_dropdown,
    dcc.Graph(id='price-graph')
])

@app.callback(
    Output(component_id='price-graph', component_property='figure'),
    Input(component_id=geo_dropdown, component_property='value')
)
def update_graph(selected_geography):
    filtered_avocado = avocado[avocado['geography'] == selected_geography]
    line_fig = px.line(filtered_avocado,
                       x='date', y='average_price',
                       color='type',
                       title=f'Avocado Prices in {selected_geography}')
    return line_fig

app.run_server(mode="jupyterlab",port=8054)

In [9]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash

df = pd.read_csv("AGB_2009-2021.csv",header=[0,1]);

df.rename(columns=         
          {'Unnamed: 0_level_0':'Temperature',
           'Unnamed: 1_level_0':'Precipitation',
           'Unnamed: 2_level_0':'Plot',
          },
          level=0,inplace=True 
)

app = JupyterDash(__name__)

geo_dropdown = dcc.Dropdown(
    options=df[('Temperature','Warmed')].unique(),value='C'
)

app.layout = html.Div(children=[
    html.H1(children='NWS-ANPP'),
    geo_dropdown,
    dcc.Graph(id='price-graph')
])

@app.callback(
    Output(component_id='price-graph', component_property='figure'),
    Input(component_id=geo_dropdown, component_property='value')
)
def update_graph(selected_tem):
    filtered_anpp = df[df[('Temperature','Warmed')] == selected_tem]
    line_fig = px.bar(
                       x=filtered_anpp[('Plot','Plot')], 
                       y=filtered_anpp[('Total','2009')],
#                        x='Plot', y=('Total','2009'),
#                        color='type',
                       title=f'ANPP in {selected_tem}'
    )
    return line_fig

app.run_server(mode="jupyterlab",port=8056)

In [12]:
app = JupyterDash(__name__)

geo_dropdown = dcc.Dropdown(
    options=['C3','C4','Total'],value='C3'
)

app.layout = html.Div(children=[
    html.H1(children='NWS-ANPP'),
    geo_dropdown,
    dcc.Graph(id='price-graph')
])

@app.callback(
    Output(component_id='price-graph', component_property='figure'),
    Input(component_id=geo_dropdown, component_property='value')
)
def update_graph(selected_tem):
    filtered_anpp = df2[selected_tem]
    line_fig = px.bar(
                       x=list(map(float,df2["C3"].index)), 
                       y=filtered_anpp,
#                        x='Plot', y=('Total','2009'),
#                        color='type',
                       title=f'ANPP in {selected_tem}'
    )
    return line_fig

app.run_server(mode="jupyterlab",port=8057)

In [396]:
# ################## design 1 ##############################
# !!!!!!!!!! design: dropdown by year !!!!!!!!!!!!!!!!!!!!

In [ ]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from itertools import product 
import plotly.graph_objects as go

In [407]:
df = pd.read_csv("AGB_2009-2021.csv",header=[0,1]);

df.rename(columns=         
          {'Unnamed: 0_level_0':'Temperature',
           'Unnamed: 1_level_0':'Precipitation',
           'Unnamed: 2_level_0':'Plot',
          },
          level=0,inplace=True 
)

header = ['Temperature', 'Precipitation', 'Plot', 'Year', 'PFT', 'Biomass']
dfx = pd.DataFrame(columns=header)
for i in range(3,df.shape[1]):
    dff1 = pd.DataFrame()
    dff1["Temperature"]   = df[('Temperature', 'Warmed')]
    dff1["Precipitation"] = df[('Precipitation',  'Water')]
    dff1["Plot"]          = df[('Plot','Plot')]
    dff1["Year"]          = df.columns[i][1]
    dff1['PFT']           = df.columns[i][0]
    dff1['Biomass']       = df[df.columns[i]]
    
    dfx = pd.concat([dfx,dff1],'rows')
    
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -     
app = JupyterDash(__name__)

geo_dropdown = dcc.Dropdown(
    options=dfx[('Year')].unique(),value="2009"
)

app.layout = html.Div(children=[
    html.H1(children='NWS-ANPP'),
    geo_dropdown,
    dcc.Graph(id='anpp-graph')
])

@app.callback(
    Output(component_id='anpp-graph', component_property='figure'),
    Input(component_id=geo_dropdown, component_property='value')
)
def update_graph(selected_tem):
    dfx1 = dfx[dfx['Year'] == selected_tem]
    
    dfx2 = dfx1.groupby(["Temperature","Precipitation","PFT"]).mean()  
    
    dfx3 = dfx2.reset_index()
    dfx3["Trmt"] = [''.join(dfx2.index[i][0:2]) for i in range(dfx2.index.shape[0])]
    dfx3.drop(dfx3[dfx3.PFT=="Total"].index,inplace=True)
    
    line_fig = px.bar(dfx3,
                       x='Trmt', 
                       y='Biomass',
                       color='PFT',
                       title=f'ANPP in {selected_tem}'
    )
    return line_fig

app.run_server(mode="jupyterlab",port=8056)

In [235]:
# ========= multiple dropdowns =================
# ===============================================

In [277]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from itertools import product 
import plotly.graph_objects as go

In [429]:
app = JupyterDash(__name__)

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    children="New warming site data dashboard",
                    className="header_title",
                ),
                html.P(
                    children="Testing ANPP",
                    className="header_description",
                ),                 
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Temperature", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="slot-filter",
                            options=[
                                {"label": Slot, "value": Slot}
                                for Slot in dfx3.Temperature.unique()
                            ],
                            multi=True,
                            value=["Select Slot(s)"],
                            clearable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Precipitation", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="dut-filter",
                            options=[
                                {"label": Dut, "value": Dut}
                                for Dut in dfx3.Precipitation.unique()
                            ],
                            value=["Select Dut(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="dut-chart",
                        config={"displayModeBar": True},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),

    ]
)

@app.callback(
    Output("dut-chart", "figure"),
    [
        Input("slot-filter", "value"),
        Input("dut-filter", "value"),
    ],
)

# def update_graph(Slot, Dut):
#     dfy = dfx3[(dfx3['Temperature'] == Slot) & (dfx3['Precipitation'] == Dut)] 
#     line_fig = px.bar(dfy,
#                        x='Year', 
#                        y='Biomass',
#                        color='PFT'
# #                        title=f'ANPP in {selected_tem}'
#     )
#     return line_fig

def update_charts(Slot, Dut):
#     mask = (
#         (data.Slot.isin(Slot))
#         & (data.Dut.isin(Dut))
#     )
#     filtered_data = data.loc[mask, :]
    
    conditions = product(Slot, Dut)
#     layout = go.Layout(barmode = 'stack')
    fig = go.Figure()
    fig.update_layout(barmode="group")
    for v in conditions:
        # print('Slot', v[0], 'Dut', v[1])
        df = dfx3[(dfx3['Temperature'] == v[0]) & (dfx3['Precipitation'] == v[1])] 
        fig.add_trace(go.Bar(
            x=df['Year'],
#             y=np.transpose(np.array([dfy[dfy.PFT=="C3"].Biomass,dfy[dfy.PFT=="C4"].Biomass])),
#             y1=df[df.PFT=="C3"].Biomass,
            y=df[df.PFT=="Total"].Biomass,
#             y =[[y1, y2]],
#             marker_color=['blue', 'red'],
            name='Temperature:{} Precipitation:{}'.format(v[0],v[1])
        ))
    
    return fig



app.run_server(mode="jupyterlab",port=8059)

In [409]:
# update with warming, precipitation, functional groups

app = JupyterDash(__name__)

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    children="New warming site data dashboard",
                    className="header_title",
                ),
                html.P(
                    children="Testing ANPP",
                    className="header_description",
                ),                 
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Temperature", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="slot-filter",
                            options=[
                                {"label": Slot, "value": Slot}
                                for Slot in dfx3.Temperature.unique()
                            ],
                            multi=True,
                            value=["Select Slot(s)"],
                            clearable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Precipitation", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="dut-filter",
                            options=[
                                {"label": Dut, "value": Dut}
                                for Dut in dfx3.Precipitation.unique()
                            ],
                            value=["Select Dut(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Plant Functional Group", 
                            className="pft-title"),
                        dcc.Dropdown(
                            id="pft-filter",
                            options=[
                                {"label": PFT, "value": PFT}
                                for PFT in dfx3.PFT.unique()
                            ],
                            value=["Select PFT(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="dut-chart",
                        config={"displayModeBar": True},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),

    ]
)

@app.callback(
    Output("dut-chart", "figure"),
    [
        Input("slot-filter", "value"),
        Input("dut-filter", "value"),
        Input("pft-filter", "value"),
    ],
)


def update_charts(Slot, Dut, PFT):
 
    conditions = product(Slot, Dut, PFT)
#     layout = go.Layout(barmode = 'stack')
    fig = go.Figure()
    fig.update_layout(barmode="group")
    for v in conditions:
        # print('Slot', v[0], 'Dut', v[1])
        df = dfx3[(dfx3['Temperature'] == v[0]) 
                  & (dfx3['Precipitation'] == v[1])
                 & (dfx3['PFT'] == v[2])] 
        fig.add_trace(go.Bar(
            x=df['Year'],
            y=df.Biomass,
            name='Warming:{} Rainfall:{} PFT:{}'.format(v[0],v[1],v[2])
        ))
    
    return fig



app.run_server(mode="jupyterlab",port=8059)
# app.run_server(mode="external",port=8080)

In [1]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from itertools import product 
import plotly.graph_objects as go

In [2]:
# multiple figures

df = pd.read_csv("AGB_2009-2021.csv",header=[0,1]);

df.rename(columns=         
          {'Unnamed: 0_level_0':'Temperature',
           'Unnamed: 1_level_0':'Precipitation',
           'Unnamed: 2_level_0':'Plot',
          },
          level=0,inplace=True 
)

header = ['Temperature', 'Precipitation', 'Plot', 'Year', 'PFT', 'Biomass']
dfx = pd.DataFrame(columns=header)
for i in range(3,df.shape[1]):
    dff1 = pd.DataFrame()
    dff1["Temperature"]   = df[('Temperature', 'Warmed')]
    dff1["Precipitation"] = df[('Precipitation',  'Water')]
    dff1["Plot"]          = df[('Plot','Plot')]
    dff1["Year"]          = df.columns[i][1]
    dff1['PFT']           = df.columns[i][0]
    dff1['Biomass']       = df[df.columns[i]]
    
    dfx = pd.concat([dfx,dff1],'rows') 
dfx2= dfx.groupby(["Temperature","Precipitation","PFT","Year"]).mean()     
dfx3 = dfx2.reset_index()
    
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -     
app = JupyterDash(__name__)

geo_dropdown = dcc.Dropdown(
    options=dfx[('Year')].unique(),value="2009"
)

app.layout = html.Div(children=[
    html.Div(children=[
    html.H1(children="New Warming Site Data Dashboard",style={'textAlign': 'center'}),
    html.H2(children='Aboveground Net Primary Productivity (ANPP: g/m2)'),
    geo_dropdown,
    dcc.Graph(id='anpp-graph')
    ]),
    
    #
    html.Div(
            children=[
                html.H1(
                    children="Aboveground Net Primary Productivity (ANPP: g/m2)",
                    className="header_title",
                ),
                html.H2(
#                     children="NWS: ANPP",
                    className="header_description",
                ),                 
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Temperature", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="slot-filter",
                            options=[
                                {"label": Slot, "value": Slot}
                                for Slot in dfx.Temperature.unique()
                            ],
                            multi=True,
                            value=["Select Slot(s)"],
                            clearable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Precipitation", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="dut-filter",
                            options=[
                                {"label": Dut, "value": Dut}
                                for Dut in dfx.Precipitation.unique()
                            ],
                            value=["Select Dut(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Plant Functional Group", 
                            className="pft-title"),
                        dcc.Dropdown(
                            id="pft-filter",
                            options=[
                                {"label": PFT, "value": PFT}
                                for PFT in dfx.PFT.unique()
                            ],
                            value=["Select PFT(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="dut-chart",
                        config={"displayModeBar": True},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    
])



@app.callback(
    Output(component_id='anpp-graph', component_property='figure'),
    Input(component_id=geo_dropdown, component_property='value')
)
def update_graph(selected_tem):
    dfx1 = dfx[dfx['Year'] == selected_tem]
    
    dfx2 = dfx1.groupby(["Temperature","Precipitation","PFT"]).mean()  
    
    dfx3 = dfx2.reset_index()
    dfx3["Trmt"] = [''.join(dfx2.index[i][0:2]) for i in range(dfx2.index.shape[0])]
    dfx3.drop(dfx3[dfx3.PFT=="Total"].index,inplace=True)
    
    line_fig = px.bar(dfx3,
                       x='Trmt', 
                       y='Biomass',
                       color='PFT',
                       title=f'ANPP in {selected_tem}'
    )
    return line_fig

@app.callback(
    Output("dut-chart", "figure"),
    [
        Input("slot-filter", "value"),
        Input("dut-filter", "value"),
        Input("pft-filter", "value"),
    ],
)


def update_charts(Slot, Dut, PFT):
 
    conditions = product(Slot, Dut, PFT)
#     layout = go.Layout(barmode = 'stack')
    fig = go.Figure()
    fig.update_layout(barmode="group")
    for v in conditions:
        # print('Slot', v[0], 'Dut', v[1])
        dfy = dfx3[(dfx3['Temperature'] == v[0]) 
                  & (dfx3['Precipitation'] == v[1])
                 & (dfx3['PFT'] == v[2])] 
        fig.add_trace(go.Bar(
            x=dfy['Year'],
            y=dfy.Biomass,
            name='Warming:{} Rainfall:{} PFT:{}'.format(v[0],v[1],v[2])
        ))
    
    return fig



app.run_server(mode="jupyterlab",port=8059)

In [420]:
# soil temperature and water content

In [5]:
dfz1 = pd.read_csv("soil_moisture.csv")
dfz2 = pd.read_csv("soil_temperature.csv")

dfz2.rename(columns= 
          {'Unnamed: 3':'2008',
           "warmign":'warm',
           'water':'precip'
          }, 
            inplace=True )

dfz2.drop(columns=['2008'],inplace=True)

header = ['Temperature', 'Precipitation', 'Plot', 'Year', 'SWC_ST','Value']
dfz = pd.DataFrame(columns=header)
for i in range(3,dfz1.shape[1]):
    dff1 = pd.DataFrame()
    dff1["Temperature"]   = dfz1['warm']
    dff1["Precipitation"] = dfz1['precip']
    dff1["Plot"]          = dfz1['plot']
    dff1["Year"]          = dfz1.columns[i]
    dff1['SWC_ST']        = 'SWC'
    dff1['Value']        = dfz1[dfz1.columns[i]]
    
    dfz = pd.concat([dfz,dff1],'rows')
    
for i in range(3,dfz2.shape[1]):
    dff1 = pd.DataFrame()
    dff1["Temperature"]   = dfz2['warm']
    dff1["Precipitation"] = dfz2['precip']
    dff1["Plot"]          = dfz2['plot']
    dff1["Year"]          = dfz2.columns[i]
    dff1['SWC_ST']        = 'ST'
    dff1['Value']        = dfz2[dfz2.columns[i]]
    
    dfz = pd.concat([dfz,dff1],'rows')


dfz3 = dfz.groupby(['Temperature', 'Precipitation',"Year","SWC_ST"]).mean()  

dfz4 = dfz3.reset_index()

dfz4.head()

,Temperature,Precipitation,Year,SWC_ST,Value
0,C,C,2009,ST,16.486000
1,C,C,2009,SWC,6.694263
2,C,C,2010,ST,16.344286
3,C,C,2010,SWC,8.165109
4,C,C,2011,ST,20.437384


In [6]:
# update with warming, precipitation, soil water/temperature

app = JupyterDash(__name__)

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    children="New warming site data dashboard",
                    className="header_title",
                ),
                html.P(
                    children="Testing ANPP",
                    className="header_description",
                ),                 
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Temperature", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="slot-filter",
                            options=[
                                {"label": Slot, "value": Slot}
                                for Slot in dfz4.Temperature.unique()
                            ],
                            multi=True,
                            value=["Select Slot(s)"],
                            clearable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Precipitation", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="dut-filter",
                            options=[
                                {"label": Dut, "value": Dut}
                                for Dut in dfz4.Precipitation.unique()
                            ],
                            value=["Select Dut(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Soil water or temperature", 
                            className="pft-title"),
                        dcc.Dropdown(
                            id="pft-filter",
                            options=[
                                {"label": SWC_ST, "value": SWC_ST}
                                for SWC_ST in dfz4.SWC_ST.unique()
                            ],
                            value=["Select PFT(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="dut-chart",
                        config={"displayModeBar": True},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),

    ]
)

@app.callback(
    Output("dut-chart", "figure"),
    [
        Input("slot-filter", "value"),
        Input("dut-filter", "value"),
        Input("pft-filter", "value"),
    ],
)


def update_charts(Slot, Dut, PFT):
 
    conditions = product(Slot, Dut, PFT)
#     layout = go.Layout(barmode = 'stack')
    fig = go.Figure()
    fig.update_layout(barmode="group")
    for v in conditions:
        # print('Slot', v[0], 'Dut', v[1])
        df = dfz4[(dfz4['Temperature'] == v[0]) 
                  & (dfz4['Precipitation'] == v[1])
                 & (dfz4['SWC_ST'] == v[2])] 
        fig.add_trace(go.Scatter(
            x=df['Year'],
            y=df.Value,
            name='Warming:{} Rainfall:{} SWC_ST:{}'.format(v[0],v[1],v[2])
        ))
    
    return fig



app.run_server(mode="jupyterlab",port=8059)
# app.run_server(mode="external",port=8080)

In [9]:
# update with warming, precipitation, soil water/temperature

app = JupyterDash(__name__)

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    children="New warming site data dashboard",
                    className="header_title",
                ),
                html.P(
                    children="Soil microclimate (SWC/ST)",
                    className="header_description",
                ),                 
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Temperature", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="tem-filter",
                            options=[
                                {"label": Temperature, "value": Temperature}
                                for Temperature in dfz4.Temperature.unique()
                            ],
                            multi=True,
                            value=["Select Temperature(s)"],
                            clearable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Precipitation", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="pre-filter",
                            options=[
                                {"label": Precipitation, "value": Precipitation}
                                for Precipitation in dfz4.Precipitation.unique()
                            ],
                            value=["Select Precipitation(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Soil water or temperature", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="swt-filter",
                            options=[
                                {"label": SWC_ST, "value": SWC_ST}
                                for SWC_ST in dfz4.SWC_ST.unique()
                            ],
                            value=["Select PFT(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="swt-chart",
                        config={"displayModeBar": True},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),

    ]
)

@app.callback(
    Output("swt-chart", "figure"),
    [
        Input("tem-filter", "value"),
        Input("pre-filter", "value"),
        Input("swt-filter", "value"),
    ],
)


def update_charts(tem, prec, swt):
 
    conditions = product(tem, prec, swt)
#     layout = go.Layout(barmode = 'stack')
    fig2 = go.Figure()
    fig2.update_layout(barmode="group")
    for v in conditions:
        # print('Slot', v[0], 'Dut', v[1])
        df = dfz4[(dfz4['Temperature'] == v[0]) 
                  & (dfz4['Precipitation'] == v[1])
                 & (dfz4['SWC_ST'] == v[2])] 
        fig2.add_trace(go.Scatter(
            x=df['Year'],
            y=df.Value,
            name='Warming:{} Rainfall:{} SWC_ST:{}'.format(v[0],v[1],v[2])
        ))
    
    return fig2



app.run_server(mode="jupyterlab",port=8059)
# app.run_server(mode="external",port=8080)

In [ ]:
# multiple figures from query database NWS.db
##* join tables 
q = "SELECT * 
    FROM SoilRespiration SR 
    JOIN SoilMoisture SM 
    ON SR.PlotID = SM.PlotID and SR.MeasurementDate = SM.MeasurementDate
    WHERE SR.Warming = 'W' and SR.Clipping = 'Yes' and SR.DeepShallow = 's'  
    and SR.MeasurementDate = 20200202
    and SM.Warming = 'W' and SM.Clipping = 'Yes' and SR.MeasurementDate = 20200202"
    
# SQLite.DBInterface.execute(NWS,q)|> DataFrame
df = DataFrame(SQLite.DBInterface.execute(NWS,q))

# scatter(df.Moisture1,df.Moisture2)

In [6]:
# multiple figures

df = pd.read_csv("AGB_2009-2021.csv",header=[0,1]);

df.rename(columns=         
          {'Unnamed: 0_level_0':'Temperature',
           'Unnamed: 1_level_0':'Precipitation',
           'Unnamed: 2_level_0':'Plot',
          },
          level=0,inplace=True 
)

header = ['Temperature', 'Precipitation', 'Plot', 'Year', 'PFT', 'Biomass']
dfx = pd.DataFrame(columns=header)
for i in range(3,df.shape[1]):
    dff1 = pd.DataFrame()
    dff1["Temperature"]   = df[('Temperature', 'Warmed')]
    dff1["Precipitation"] = df[('Precipitation',  'Water')]
    dff1["Plot"]          = df[('Plot','Plot')]
    dff1["Year"]          = df.columns[i][1]
    dff1['PFT']           = df.columns[i][0]
    dff1['Biomass']       = df[df.columns[i]]
    
    dfx = pd.concat([dfx,dff1],'rows') 
dfx2= dfx.groupby(["Temperature","Precipitation","PFT","Year"]).mean()     
dfx3 = dfx2.reset_index()

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
dfz1 = pd.read_csv("soil_moisture.csv")
dfz2 = pd.read_csv("soil_temperature.csv")

dfz2.rename(columns= 
          {'Unnamed: 3':'2008',
           "warmign":'warm',
           'water':'precip'
          }, 
            inplace=True )

dfz2.drop(columns=['2008'],inplace=True)

header = ['Temperature', 'Precipitation', 'Plot', 'Year', 'SWC_ST','Value']
dfz = pd.DataFrame(columns=header)
for i in range(3,dfz1.shape[1]):
    dff1 = pd.DataFrame()
    dff1["Temperature"]   = dfz1['warm']
    dff1["Precipitation"] = dfz1['precip']
    dff1["Plot"]          = dfz1['plot']
    dff1["Year"]          = dfz1.columns[i]
    dff1['SWC_ST']        = 'SWC'
    dff1['Value']        = dfz1[dfz1.columns[i]]
    
    dfz = pd.concat([dfz,dff1],'rows')
    
for i in range(3,dfz2.shape[1]):
    dff1 = pd.DataFrame()
    dff1["Temperature"]   = dfz2['warm']
    dff1["Precipitation"] = dfz2['precip']
    dff1["Plot"]          = dfz2['plot']
    dff1["Year"]          = dfz2.columns[i]
    dff1['SWC_ST']        = 'ST'
    dff1['Value']        = dfz2[dfz2.columns[i]]
    
    dfz = pd.concat([dfz,dff1],'rows')


dfz3 = dfz.groupby(['Temperature', 'Precipitation',"Year","SWC_ST"]).mean()  

dfz4 = dfz3.reset_index()
    
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -     
app = JupyterDash(__name__)

geo_dropdown = dcc.Dropdown(
    options=dfx[('Year')].unique(),value="2009"
)

app.layout = html.Div(children=[
    html.Div(children=[
    html.H1(children="New Warming Site Data Dashboard",style={'textAlign': 'center'}),
    html.H2(children='Aboveground Net Primary Productivity (ANPP: g/m2)'),
    geo_dropdown,
    dcc.Graph(id='anpp-graph')
    ]),
    
    #
    html.Div(
            children=[
                html.H1(
                    children="Aboveground Net Primary Productivity (ANPP: g/m2)",
                    className="header_title",
                ),
                html.H2(
#                     children="NWS: ANPP",
                    className="header_description",
                ),                 
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Temperature", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="slot-filter",
                            options=[
                                {"label": Slot, "value": Slot}
                                for Slot in dfx.Temperature.unique()
                            ],
                            multi=True,
                            value=["Select Slot(s)"],
                            clearable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Precipitation", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="dut-filter",
                            options=[
                                {"label": Dut, "value": Dut}
                                for Dut in dfx.Precipitation.unique()
                            ],
                            value=["Select Dut(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Plant Functional Group", 
                            className="pft-title"),
                        dcc.Dropdown(
                            id="pft-filter",
                            options=[
                                {"label": PFT, "value": PFT}
                                for PFT in dfx.PFT.unique()
                            ],
                            value=["Select PFT(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="dut-chart",
                        config={"displayModeBar": True},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    #
    #
    html.Div(
            children=[
                html.H1(
                    children="Soil microclimate (SWC/ST)",
                    className="header_title",
                ),
                html.P(
#                     children="Soil microclimate (SWC/ST)",
                    className="header_description",
                ),                 
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Temperature", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="tem-filter",
                            options=[
                                {"label": Temperature, "value": Temperature}
                                for Temperature in dfz4.Temperature.unique()
                            ],
                            multi=True,
                            value=["Select Temperature(s)"],
                            clearable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Precipitation", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="pre-filter",
                            options=[
                                {"label": Precipitation, "value": Precipitation}
                                for Precipitation in dfz4.Precipitation.unique()
                            ],
                            value=["Select Precipitation(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Soil water or temperature", 
                            className="menu-title"),
                        dcc.Dropdown(
                            id="swt-filter",
                            options=[
                                {"label": SWC_ST, "value": SWC_ST}
                                for SWC_ST in dfz4.SWC_ST.unique()
                            ],
                            value=["Select PFT(s)"],
                            clearable=False,
                            searchable=False,
                            multi=True,
                            className="dropdown",
                        ),
                    ],
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="swt-chart",
                        config={"displayModeBar": True},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        )
    
    
])



@app.callback(
    Output(component_id='anpp-graph', component_property='figure'),
    Input(component_id=geo_dropdown, component_property='value')
)
def update_graph(selected_tem):
    dfx1 = dfx[dfx['Year'] == selected_tem]
    
    dfx2 = dfx1.groupby(["Temperature","Precipitation","PFT"]).mean()  
    
    dfx3 = dfx2.reset_index()
    dfx3["Trmt"] = [''.join(dfx2.index[i][0:2]) for i in range(dfx2.index.shape[0])]
    dfx3.drop(dfx3[dfx3.PFT=="Total"].index,inplace=True)
    
    line_fig = px.bar(dfx3,
                       x='Trmt', 
                       y='Biomass',
                       color='PFT',
                       title=f'ANPP in {selected_tem}'
    )
    return line_fig

@app.callback(
    Output("dut-chart", "figure"),
    [
        Input("slot-filter", "value"),
        Input("dut-filter", "value"),
        Input("pft-filter", "value"),
    ],
)


def update_charts(Slot, Dut, PFT):
 
    conditions = product(Slot, Dut, PFT)
#     layout = go.Layout(barmode = 'stack')
    fig = go.Figure()
    fig.update_layout(barmode="group")
    for v in conditions:
        # print('Slot', v[0], 'Dut', v[1])
        dfy = dfx3[(dfx3['Temperature'] == v[0]) 
                  & (dfx3['Precipitation'] == v[1])
                 & (dfx3['PFT'] == v[2])] 
        fig.add_trace(go.Bar(
            x=dfy['Year'],
            y=dfy.Biomass,
            name='Warming:{} Rainfall:{} PFT:{}'.format(v[0],v[1],v[2])
        ))
    
    return fig

#
@app.callback(
    Output("swt-chart", "figure"),
    [
        Input("tem-filter", "value"),
        Input("pre-filter", "value"),
        Input("swt-filter", "value"),
    ],
)
def update_charts2(tem, prec, swt):
 
    conditions = product(tem, prec, swt)
#     layout = go.Layout(barmode = 'stack')
    fig2 = go.Figure()
    fig2.update_layout()
    for v in conditions:
        # print('Slot', v[0], 'Dut', v[1])
        df = dfz4[(dfz4['Temperature'] == v[0]) 
                  & (dfz4['Precipitation'] == v[1])
                 & (dfz4['SWC_ST'] == v[2])] 
        fig2.add_trace(go.Scatter(
            x=df['Year'],
            y=df.Value,
            name='Warming:{} Rainfall:{} SWC_ST:{}'.format(v[0],v[1],v[2])
        ))
    
    return fig2



# app.run_server(mode="jupyterlab",port=8059)
app.run_server(mode="external",port=8059)

Dash app running on http://127.0.0.1:8059/
